<a href="https://colab.research.google.com/github/nd-thang/-KPDL20-21-Sentiment-analysis-hotel-review/blob/main/KPDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nd-thang/-KPDL20-21-Sentiment-analysis-hotel-review.git data

Cloning into 'data'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


###Count each label

In [ ]:
filename = '/data/sentiment_analysis_train.v1.0.txt'
f = open(filename, "r", encoding="utf8")
lines = f.readlines()
labels = []
reviews =[]
for line in lines:
    l = line.strip().split(' ', 1)
    labels.append(l[0])
    reviews.append(l[1])

print(len(labels))
print(len(reviews))

print(labels.count(''))
print(labels.count(' '))
print(reviews.count(''))
print(reviews.count(' '))
for label in set(labels):
    print(labels.count(label))
    print(label)

22999
22999
0
0
0
0
214
__label__rat_kem
10735
__label__tot
4569
__label__trung_binh
6396
__label__xuat_sac
1085
__label__kem


###Translate train and test file to lower english

In [ ]:
pip install google-trans-new

In [ ]:
from google_trans_new import google_translator  

translator = google_translator()
print(translator.translate('anh yeu em'))


I love you 


In [ ]:
#file train
from google_trans_new import google_translator
f = open("/data/en_train_fin_lower.txt", "w")
translator = google_translator()
filename = '/data/sentiment_analysis_train.v1.0.txt'
f2 = open(filename, "r", encoding="utf8")
lines = f2.readlines()
#lines2 = lines[:10]
for line in lines:
  l = line.strip().split(' ', 1)
  f.write(l[0] + ' ' + translator.translate(l[1]).lower() + "\n")
  #f2.write(translator.translate(line).lower() + "\n")
f2.close()
f.close()


In [ ]:
#file test
f = open("/data/en_test_fin_lower.txt", "w")
filename = '/data/sentiment_analysis_test_unlabel.v1.0.txt'
f2 = open(filename, "r", encoding="utf8")
lines = f2.readlines()
#lines2 = lines[:10]
for line in lines:
  f.write(translator.translate(line).lower() + "\n")
f2.close()
f.close()

###Create labels, reviews, tests lists

In [ ]:
filename = "/data/en_train_fin_lower.txt"
f = open(filename, "r", encoding="utf8")
lines = f.readlines()
labels = []
reviews =[]
for line in lines:
    l = line.strip().split(' ', 1)
    labels.append(l[0])
    reviews.append(l[1])

In [ ]:
filename = '/data/en_test_fin_lower.txt'
f = open(filename, "r", encoding="utf8")
lines = f.readlines()
tests =[]
for line in lines:
    tests.append(line)
f.close()

In [ ]:
pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=19)


###LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import time
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs', 
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
text_clf = text_clf.fit(X_train, y_train)
 
train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

Done training Linear Classifier in 6.67122220993042 seconds.


In [ ]:
y_pred = text_clf.predict(X_test)
text_clf.score(X_test, y_test)

0.6704347826086956

In [ ]:
print(len(y_pred))
print(len(X_test))

4600
4600


In [ ]:
#kfold LR
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import time
import numpy as np

from sklearn.model_selection import KFold

scores = []
kf = KFold(n_splits=5, shuffle=True)
np_reviews = np.array(reviews)
np_labels = np.array(labels)
for train_index, test_index in kf.split(np_reviews):
    X_train, X_test = np_reviews[train_index], np_reviews[test_index]
    y_train, y_test = np_labels[train_index], np_labels[test_index]
    model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs', 
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
print(scores)
print(np.mean(scores))
final_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs', 
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
final_model.fit(np_reviews, np_labels)



[0.6617391304347826, 0.6689130434782609, 0.6702173913043479, 0.6682608695652174, 0.6642748423570342]
0.6666810554279285


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.8,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

###SVM

In [ ]:
from sklearn.svm import SVC
 
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
text_clf = text_clf.fit(X_train, y_train)
 
train_time = time.time() - start_time
print('Done training SVM in', train_time, 'seconds.')

Done training SVM in 150.69980096817017 seconds.


In [ ]:
y_pred = text_clf.predict(X_test)
text_clf.score(X_test, y_test)

0.6743478260869565

In [ ]:
#svm kfold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import time
import numpy as np
from sklearn.svm import SVC

from sklearn.model_selection import KFold

scores = []
kf = KFold(n_splits=5, shuffle=True)
np_reviews = np.array(reviews)
np_labels = np.array(labels)
for train_index, test_index in kf.split(np_reviews):
    X_train, X_test = np_reviews[train_index], np_reviews[test_index]
    y_train, y_test = np_labels[train_index], np_labels[test_index]
    model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
print(scores)
print(np.mean(scores))
final_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
#final_model.fit(np_reviews, np_labels)


[0.6797826086956522, 0.672608695652174, 0.6580434782608696, 0.6680434782608695, 0.6677538595346815]
0.6692464240808493


###Naive bayes

In [ ]:
#naive bayes
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())
                    ])
text_clf = text_clf.fit(X_train, y_train)
 
train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')

Done training Naive Bayes in 0.8197929859161377 seconds.


In [ ]:
y_pred = text_clf.predict(X_test)
text_clf.score(X_test, y_test)

0.5927375516416612

In [ ]:
#naive bayes kfold
#kfold LR
scores = []
kf = KFold(n_splits=5, shuffle=True)
np_reviews = np.array(reviews)
np_labels = np.array(labels)
for train_index, test_index in kf.split(np_reviews):
    X_train, X_test = np_reviews[train_index], np_reviews[test_index]
    y_train, y_test = np_labels[train_index], np_labels[test_index]
    model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())
                    ])
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
print(scores)
print(np.mean(scores))
final_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())
                    ])
#final_model.fit(X, y)


[0.6115217391304347, 0.5960869565217392, 0.596304347826087, 0.6065217391304348, 0.5979560774081322]
0.6016781720033656


### Create result label file

In [ ]:
#using final model to predict
y_pred = final_model.predict(tests)
#create result label file
filename = '/data/LR_res.txt'
f = open(filename, "w", encoding="utf8")
for line in y_pred:
    f.write(line + "\n")
f.close()

###Fasttext

In [ ]:
#fasttext
!pip3 install numpy scipy pybind11 cmake
!wget -nc "https://github.com/facebookresearch/fastText/archive/v0.9.2.zip" > /dev/null
!unzip -n "v0.9.2.zip" > /dev/null
!cd fastText-0.9.2 && pip3 install .
!rm -rf v0.9.2.zip fastText-0.9.2

     |████████████████████████████████| 194kB 5.8MB/s 
--2020-12-06 15:32:59--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2020-12-06 15:33:00--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  16.6MB/s    in 0.3s    

2020-12-06 15:33:00 (16.6 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Processing /content/fastText-0.9.2
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36

In [ ]:
#create train and test file for fasttext
filename = '/data/train_fasttext.txt'
f = open(filename, "w")
for i in range(len(X_train)):
    f.write(y_train[i] + ' ' + X_train[i] + "\n")
f.close()
filename = '/data/test_fasttext.txt'
f = open(filename, "w")
for i in range(len(X_test)):
    f.write(y_test[i] + ' ' + X_test[i] + "\n")
f.close()

In [ ]:
import fasttext
 
start_time = time.time()
model = fasttext.train_supervised(
                                input='/data/train_fasttext.txt',
                                #input='/data/en_train_fin_lower.txt',
                                dim=100,
                                epoch=8,
                                lr=0.1,
                                wordNgrams=2,
                                label='__label__',
                                minCount=5
)
 
train_time = time.time() - start_time
print('Done training Fasttext in', train_time, 'seconds.')

Done training Fasttext in 3.7627036571502686 seconds.


In [ ]:
def print_results(N, p, r):
    print("Fasttext, Precision = {}, Recall = {}".format(p, r))

print_results(*model.test('/data/test_fasttext.txt'))

Fasttext, Precision = 0.7027614698847575, Recall = 0.7027614698847575


In [ ]:
#kfold fastext
%%time
import fasttext

np_reviews = np.array(reviews)
np_labels = np.array(labels)
scores = []
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(np_reviews):
    X_train, X_test = np_reviews[train_index], np_reviews[test_index]
    y_train, y_test = np_labels[train_index], np_labels[test_index]
    #create train and test file for fasttext
    filename = '/data/train_fasttext.txt'
    f = open(filename, "w")
    for i in range(len(X_train)):
        f.write(y_train[i] + ' ' + X_train[i] + "\n")
    f.close()
    filename = '/data/test_fasttext.txt'
    f = open(filename, "w")
    for i in range(len(X_test)):
        f.write(y_test[i] + ' ' + X_test[i] + "\n")
    f.close()
    model = fasttext.train_supervised(
                                input='/data/train_fasttext.txt',
                                #input='/data/en_train_fin_lower.txt',
                                dim=100,
                                epoch=8,
                                lr=0.1,
                                wordNgrams=2,
                                label='__label__',
                                minCount=5)
    scores.append(model.test('/data/test_fasttext.txt')[1])
print(scores)
print(np.mean(scores))


[0.7215217391304348, 0.6941304347826087, 0.6965217391304348, 0.6984782608695652, 0.7040661013263753]
0.7029436550478838
CPU times: user 20.3 s, sys: 1.31 s, total: 21.6 s
Wall time: 21.6 s


###Finally write fasttext predicts to file

In [ ]:
filename = '/data/fat_res_fin.txt'
f = open(filename, "w", encoding="utf8")
f2 = open("/data/en_test_fin_lower.txt", "r")
lines = f2.readlines()
#lines2 = lines[:10]
for line in lines:
    s = model.predict(line.strip()) 
    f.write(s[0][0] + "\n")
f.close()
f2.close()

###Translate en file to vi and tokenize

In [ ]:
#file train
%%time
from google_trans_new import google_translator
f = open("/data/vi_train_from_en_tokenized.txt", "w")
translator = google_translator()
filename = '/data/en_train_fin_lower.txt'
f2 = open(filename, "r", encoding="utf8")
lines = f2.readlines()
#lines2 = lines[:5]
for line in lines:
  l = line.strip().split(' ', 1)
  f.write(l[0] + ' ' + word_tokenize(translator.translate(l[1],'vi').lower(), format = 'text') + "\n")
  #f2.write(translator.translate(line).lower() + "\n")
f2.close()
f.close()

CPU times: user 2min 48s, sys: 8.29 s, total: 2min 56s
Wall time: 59min 8s


In [ ]:
#file test
%%time
f = open("/data/vi_test_from_en_tokenized.txt", "a")
filename = '/data/en_test_fin_lower.txt'
f2 = open(filename, "r", encoding="utf8")
lines = f2.readlines()
lines2 = lines[9471:]
for line in lines2:
  f.write(word_tokenize(translator.translate(line,'vi').lower(), format = 'text') + "\n")
f2.close()
f.close()

CPU times: user 1min 20s, sys: 3.48 s, total: 1min 23s
Wall time: 28min 11s


###GridSearch 

In [ ]:
#Todo